In [215]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsRegressor

In [216]:
def standard_scale(x):
    res = (x - x.mean(axis = 0)) / x.std(axis = 0)
    return res

In [217]:
X, y = datasets.load_iris(return_X_y = True)
X = iris.data
# X.shape

X = X.astype(float)
X = standard_scale(X)

### Задание 1

#### Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [218]:
# функция понижения размерности методом PCA
def get_new_features_PCA(X, n_features):
    
    # Найдем собственные векторы и собственные значения
    #------------------------------------------------------------------------------------------------------------ 
    covariance_matrix = X.T @ X

    eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

    # сформируем список кортежей (собственное значение, собственный вектор)
    eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

    # и отсортируем список по убыванию собственных значений
    eig_pairs.sort(key=lambda x: x[0], reverse = True)

    print('\nСобственные значения и собственные векторы в порядке убывания:')
    for i in eig_pairs:
        print(i)

    #------------------------------------------------------------------------------------------------------------
    # Оценим долю дисперсии, которая описывается найденными компонентами.
    eig_sum = sum(eig_values)
    var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse = True)]
    cum_var_exp = np.cumsum(var_exp)
    
    print(f'\nДоля дисперсии, описываемая каждой из компонент \n{var_exp}')

    # а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
    print(f'\nКумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

    #------------------------------------------------------------------------------------------------------------
    # Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
    W = np.hstack([eig_pairs[i][1].reshape(4, 1) for i in range(n_features)])

    print(f'\nМатрица весов W:\n', W)

    #------------------------------------------------------------------------------------------------------------
    #print(f'\nX\n', X[:10, :])
    #------------------------------------------------------------------------------------------------------------
    # Сформируем новую матрицу "объекты-признаки"
    Z = X.dot(W)
    
    return Z

In [219]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 21)

In [220]:
#--------------------------------------------------------------------------------------------------------
# Модель на четырех признаках
#--------------------------------------------------------------------------------------------------------
model = KNeighborsRegressor(3)

model.fit(X_train, y_train)

y_train_preds = model.predict(X_train).astype(int)
y_test_preds  = model.predict(X_test).astype(int)

print('4 признака')
accuracy_4_train = round(accuracy_score(y_train, y_train_preds), 4)
accuracy_4_test  = round(accuracy_score(y_test, y_test_preds), 4)

print(f'\tAccuracy Train =', accuracy_4_train)
print(f'\tAccuracy Test  =', accuracy_4_test)

4 признака
	Accuracy Train = 0.95
	Accuracy Test  = 0.9333


In [221]:
# Понижение размерности до двух признаков
Z_pca = get_new_features_PCA(X, 2)


Собственные значения и собственные векторы в порядке убывания:
(437.77467247979894, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.10457072021055, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.01353133569725, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.1072254642929513, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329986, 22.85076178670177, 3.668921889282877, 0.5178709107154922]

Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [222]:
#--------------------------------------------------------------------------------------------------------
# Модель на двух признаках
#--------------------------------------------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(Z_pca, y, test_size = 0.2, random_state = 21)

model.fit(X_train, y_train)

y_train_preds = model.predict(X_train).astype(int)
y_test_preds  = model.predict(X_test).astype(int)

print('2 признака')
accuracy_2_train = round(accuracy_score(y_train, y_train_preds), 4)
accuracy_2_test  = round(accuracy_score(y_test, y_test_preds), 4)

print(f'\tAccuracy Train =', accuracy_2_train)
print(f'\tAccuracy Test  =', accuracy_2_test)

2 признака
	Accuracy Train = 0.8833
	Accuracy Test  = 0.8667


### Вывод
После понижения размерности с четырех до двух признаков качество модели на тестовой выборке снижается с 93% до 86%, но в целом остается достаточно хорошим.

### Задание 2

#### Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [223]:
X, y = datasets.load_iris(return_X_y = True)
X = iris.data
# X.shape

X = X.astype(float)
X = standard_scale(X)

In [224]:
# функция понижения размерности методом SVD
def get_new_features_SVD(X, n_features):
    
    # Найдем собственные векторы и собственные значения
    #------------------------------------------------------------------------------------------------------------ 
    covariance_matrix = X.T @ X

    #------------------------------------------------------------------------------------------------------------ 
    # разложение матрицы ковариации
    U, D, VT = np.linalg.svd(covariance_matrix)
    
    eig_values  = D
    eig_vectors = VT.T
    #------------------------------------------------------------------------------------------------------------ 

    # сформируем список кортежей (собственное значение, собственный вектор)
    eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

    # и отсортируем список по убыванию собственных значений
    eig_pairs.sort(key=lambda x: x[0], reverse = True)

    print('\nСобственные значения и собственные векторы в порядке убывания:')
    for i in eig_pairs:
        print(i)

    #------------------------------------------------------------------------------------------------------------
    # Оценим долю дисперсии, которая описывается найденными компонентами.
    eig_sum = sum(eig_values)
    var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse = True)]
    cum_var_exp = np.cumsum(var_exp)

    print(f'\nДоля дисперсии, описываемая каждой из компонент \n{var_exp}')

    # а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
    print(f'\nКумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

    #------------------------------------------------------------------------------------------------------------
    # Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
    W = np.hstack([eig_pairs[i][1].reshape(4, 1) for i in range(n_features)])

    print(f'\nМатрица весов W:\n', W)

    #------------------------------------------------------------------------------------------------------------
    #print(f'\nX\n', X[:10, :])
    #------------------------------------------------------------------------------------------------------------
    # Сформируем новую матрицу "объекты-признаки"
    Z = X.dot(W)

    return Z

In [225]:
# Понижение размерности до двух признаков
Z_svd = get_new_features_SVD(X, 2)


Собственные значения и собственные векторы в порядке убывания:
(437.77467247979934, array([-0.52106591,  0.26934744, -0.5804131 , -0.56485654]))
(137.10457072021055, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.013531335697266, array([ 0.71956635, -0.24438178, -0.14212637, -0.63427274]))
(3.107225464292947, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329987, 22.850761786701753, 3.668921889282877, 0.5178709107154911]

Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]

Матрица весов W:
 [[-0.52106591 -0.37741762]
 [ 0.26934744 -0.92329566]
 [-0.5804131  -0.02449161]
 [-0.56485654 -0.06694199]]


In [226]:
#--------------------------------------------------------------------------------------------------------
# Проверка Модели на двух признаках
#--------------------------------------------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(Z_svd, y, test_size = 0.2, random_state = 21)

model.fit(X_train, y_train)

y_train_preds = model.predict(X_train).astype(int)
y_test_preds  = model.predict(X_test).astype(int)

print('2 признака')
accuracy_2_train = round(accuracy_score(y_train, y_train_preds), 4)
accuracy_2_test  = round(accuracy_score(y_test, y_test_preds), 4)

print(f'\tAccuracy Train =', accuracy_2_train)
print(f'\tAccuracy Test  =', accuracy_2_test)

2 признака
	Accuracy Train = 0.8833
	Accuracy Test  = 0.8667


### Вывод 
Значения Accuracy для train и для test для двух признаков получились такие же, как и в задании 1.

Оба метода понижения размерности работают одинаково.